In [1]:
import argparse
import os

import cv2
import math
import numpy as np
import moviepy 
from matplotlib import pyplot as plt


import torch

from alphaction.config import cfg
from alphaction.dataset import make_data_loader
from alphaction.engine.inference import inference
from alphaction.modeling.detector import build_detection_model
from alphaction.utils.checkpoint import ActionCheckpointer
from torch.utils.collect_env import get_pretty_env_info
from alphaction.utils.comm import synchronize, get_rank
from alphaction.utils.logger import setup_logger
#pytorch issuse #973
import resource

from my_utils.gen_utils import create_experiment_folder
from tqdm import tqdm


### 1. CONFIG
#### 1.1 Main Parameters

In [2]:
model_name = 'VMAEv2'


person_threshold = 0.6 # confidence threshold on actor. 0.6 is the defualt
sampling_rate = 3 # sampling rate: 4 is the defualt
top_k = 5 # number of actions per person
video_path = '../input_dir/markt2_fight.mp4'

slice_height = 800
slice_width = 1000
overlap_ratio = 0.1

starting_frame_index = 0
length_input = 368

exp_dict = {'model_name': model_name,
            'model_params': {'person_threshold': person_threshold, 
                             'sampling_rate': sampling_rate},
            'orig_post_processing':{'top_k': top_k},
            'aggregation': {'method': {}, 
                            'params': {}},
            'video_path': video_path,
            'slicing_params': {'slice_height': slice_height, 
                               'slice_width': slice_width, 
                               'overlap_ratio':overlap_ratio},
            'video_params': {'st_frame_index': starting_frame_index, 
                             'length_input':length_input
                             }
           }



In [3]:
video_name = os.path.basename(video_path).split('.')[0]
output_directory = f'../output_dir/{video_name}/{model_name}/patch_batch/' 
output_directory = create_experiment_folder(output_directory, 'exp')
output_directory



'../output_dir/markt2_fight/VMAEv2/patch_batch/exp_1'

#### 1.2 Model Config Setup

In [4]:
if model_name == 'VMAEv2':
    config_file = '../config_files/VMAEv2-ViTB-16x4.yaml'
if model_name == 'VMAE':
    config_file = '../config_files/VMAE-ViTB-16x4.yaml'


In [5]:
cfg.merge_from_file(config_file)

In [6]:
# change model weight path
if model_name == 'VMAEv2':
    cfg.merge_from_list(["MODEL.WEIGHT", "../checkpoints/VMAEv2_ViTB_16x4.pth"])
if model_name == 'VMAE':
    cfg.merge_from_list(["MODEL.WEIGHT", "../checkpoints/VMAE_ViTB_16x4.pth"])

# change output dir
cfg.merge_from_list(["OUTPUT_DIR", "../output_dir/"])

# change person threshold
cfg.merge_from_list(["MODEL.STM.PERSON_THRESHOLD", person_threshold])

# change sampling rate
cfg.merge_from_list(["DATA.SAMPLING_RATE", sampling_rate])

# change path for data_dir
cfg.merge_from_list(["DATA.PATH_TO_DATA_DIR", "/work/ava"])

# folder name of annotations
cfg.merge_from_list(["AVA.ANNOTATION_DIR", "annotations/"])

# file name of  frame_lists
cfg.merge_from_list(["AVA.TRAIN_LISTS", ['sample.csv']])
cfg.merge_from_list(["AVA.TEST_LISTS", ['sample.csv']])

# file name of predicted_bboxes
cfg.merge_from_list(["AVA.TRAIN_GT_BOX_LISTS", ['ava_sample_predicted_boxes.csv']])
cfg.merge_from_list(["AVA.TEST_GT_BOX_LISTS", ['ava_sample_predicted_boxes.csv']])

# file name of exlusions
cfg.merge_from_list(["AVA.EXCLUSION_FILE", 'ava_sample_train_excluded_timestamps_v2.2.csv'])

# number of batches in test scenario
cfg.merge_from_list(["TEST.VIDEOS_PER_BATCH", 1])

# number of workers
cfg.merge_from_list(["DATALOADER.NUM_WORKERS", 1])


### 2. ARGS

In [7]:
debug = True
if debug:
    # The shape of model input should be divisible into this. Otherwise, padding 0 to left and bottum. 
    print("cfg.DATALOADER.SIZE_DIVISIBILITY: ", cfg.DATALOADER.SIZE_DIVISIBILITY)
    
    # Sampling rate in constructing the clips.
    self_sample_rate =  cfg.DATA.SAMPLING_RATE
    print("cfg.DATA.SAMPLING_RATE: ", cfg.DATA.SAMPLING_RATE)
    
    # Length of clip
    self_video_length = cfg.DATA.NUM_FRAMES
    print("cfg.DATA.NUM_FRAMES: ", cfg.DATA.NUM_FRAMES)
    
    # Length of sequence frames from which a clip is constructed.
    self_seq_len = self_video_length * self_sample_rate
    print("self_seq_len: ", self_seq_len)
    
    self_num_classes = cfg.MODEL.STM.ACTION_CLASSES
    print("cfg.MODEL.STM.ACTION_CLASSES: ", self_num_classes)
    
    # Augmentation params.
    self_data_mean = cfg.DATA.MEAN
    self_data_std = cfg.DATA.STD
    self_use_bgr = cfg.AVA.BGR
    print("Augmentation params: ", self_data_mean, self_data_std, self_use_bgr)
    
    self_jitter_min_scale = cfg.DATA.TEST_MIN_SCALES
    self_jitter_max_scale = cfg.DATA.TEST_MAX_SCALE
    self_test_force_flip = cfg.AVA.TEST_FORCE_FLIP

    print("scale and flip params", self_jitter_min_scale, self_jitter_max_scale, self_test_force_flip)

cfg.DATALOADER.SIZE_DIVISIBILITY:  32
cfg.DATA.SAMPLING_RATE:  3
cfg.DATA.NUM_FRAMES:  16
self_seq_len:  48
cfg.MODEL.STM.ACTION_CLASSES:  80
Augmentation params:  [0.45, 0.45, 0.45] [0.225, 0.225, 0.225] False
scale and flip params [256] 1333 False


In [8]:
seq_len = cfg.DATA.NUM_FRAMES * cfg.DATA.SAMPLING_RATE
print(seq_len)

48


### 3. VIDEO Info and Slicing Visualization


In [9]:
from my_utils.video_processing import get_video_info, get_frame_from_video

In [10]:
video_info = get_video_info(video_path)

In [11]:
video_info

{'frame_count': 368,
 'frame_rate': 16.999,
 'width': 2592,
 'height': 1944,
 'fps': 16.999}

In [12]:
exp_dict['video_params'].update(video_info)
exp_dict

{'model_name': 'VMAEv2',
 'model_params': {'person_threshold': 0.6, 'sampling_rate': 3},
 'orig_post_processing': {'top_k': 5},
 'aggregation': {'method': {}, 'params': {}},
 'video_path': '../input_dir/markt2_fight.mp4',
 'slicing_params': {'slice_height': 800,
  'slice_width': 1000,
  'overlap_ratio': 0.1},
 'video_params': {'st_frame_index': 0,
  'length_input': 368,
  'frame_count': 368,
  'frame_rate': 16.999,
  'width': 2592,
  'height': 1944,
  'fps': 16.999}}

In [13]:
frame_height = video_info['height']
frame_width = video_info['width']

In [14]:
from my_utils.slicing import get_slice_bboxes, generate_sliding_window_gif

In [15]:
patches_coordinates = get_slice_bboxes(frame_height, frame_width, slice_height, slice_width, False, overlap_ratio)

In [16]:
len(patches_coordinates)

9

In [17]:
patches_coordinates

[[0, 0, 1000, 800],
 [900, 0, 1900, 800],
 [1592, 0, 2592, 800],
 [0, 720, 1000, 1520],
 [900, 720, 1900, 1520],
 [1592, 720, 2592, 1520],
 [0, 1144, 1000, 1944],
 [900, 1144, 1900, 1944],
 [1592, 1144, 2592, 1944]]

In [18]:
sample_frame = get_frame_from_video(video_path, 0)

In [19]:
if False:
    generate_sliding_window_gif(sample_frame, patches_coordinates, gif_filename='sliding_window.gif')

In [20]:
from IPython.display import display, Image


In [21]:
if False:
    display(Image(filename='sliding_window.gif'))

In [22]:
if True:
    !rm sliding_window.gif

rm: cannot remove 'sliding_window.gif': No such file or directory


### 4. building model

In [23]:
model = build_detection_model(cfg)

In [24]:
model.to("cuda")

STMDetector(
  (backbone): ViT(
    (patch_embed): PatchEmbed(
      (proj): Conv3d(3, 768, kernel_size=(2, 16, 16), stride=(2, 16, 16))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0): Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=False)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
        )
      )
      (1): Block(
        (norm1): La

In [25]:
model.__dict__

{'training': True,
 '_parameters': OrderedDict(),
 '_buffers': OrderedDict(),
 '_non_persistent_buffers_set': set(),
 '_backward_pre_hooks': OrderedDict(),
 '_backward_hooks': OrderedDict(),
 '_is_full_backward_hook': None,
 '_forward_hooks': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_state_dict_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict(),
 '_load_state_dict_post_hooks': OrderedDict(),
 '_modules': OrderedDict([('backbone',
               ViT(
                 (patch_embed): PatchEmbed(
                   (proj): Conv3d(3, 768, kernel_size=(2, 16, 16), stride=(2, 16, 16))
                 )
                 (pos_drop): Dropout(p=0.0, inplace=False)
                 (blocks): ModuleList(
                   (0): Block(
                     (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
                     (attn): Attention(
                       (qkv): Linear(in_features=768, out_features=2304, bias=False)
                       (att

### 5. loading weight

In [26]:
output_dir = cfg.OUTPUT_DIR
output_dir

'../output_dir/'

In [27]:
checkpointer = ActionCheckpointer(cfg, model, save_dir=output_dir)
checkpointer.load(cfg.MODEL.WEIGHT)

{}

In [28]:
num_gpus = int(os.environ["WORLD_SIZE"]) if "WORLD_SIZE" in os.environ else 1
num_gpus

1

In [29]:
mem_active = cfg.MODEL.STM.MEM_ACTIVE
mem_active  

False

In [30]:
from alphaction.utils.comm import get_world_size

In [31]:
device = torch.device("cuda")

In [32]:
num_devices = get_world_size()
num_devices

1

In [33]:
model.eval()

STMDetector(
  (backbone): ViT(
    (patch_embed): PatchEmbed(
      (proj): Conv3d(3, 768, kernel_size=(2, 16, 16), stride=(2, 16, 16))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0): Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=False)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
        )
      )
      (1): Block(
        (norm1): La

## 6. Inference

In [34]:
patches_coordinates

[[0, 0, 1000, 800],
 [900, 0, 1900, 800],
 [1592, 0, 2592, 800],
 [0, 720, 1000, 1520],
 [900, 720, 1900, 1520],
 [1592, 720, 2592, 1520],
 [0, 1144, 1000, 1944],
 [900, 1144, 1900, 1944],
 [1592, 1144, 2592, 1944]]

In [35]:
from my_utils.video_processing import segment_crop_video

In [36]:
from my_utils.my_ava_preprocessing import ava_preprocessing_cv2, clip_constructor, prepare_collated_batches


In [37]:
from my_utils.gen_utils import parse_label_file

In [38]:
file_path = 'labels.txt'  # Specify the path to your text file
label_dict = parse_label_file(file_path)
print(label_dict)

{1: 'bend/bow (at the waist)', 2: 'crawl', 3: 'crouch/kneel', 4: 'dance', 5: 'fall down', 6: 'get up', 7: 'jump/leap', 8: 'lie/sleep', 9: 'martial art', 10: 'run/jog', 11: 'sit', 12: 'stand', 13: 'swim', 14: 'walk', 15: 'answer phone', 16: 'brush teeth', 17: 'carry/hold (an object)', 18: 'catch (an object)', 19: 'chop', 20: 'climb (e.g., a mountain)', 21: 'clink glass', 22: 'close (e.g., a door, a box)', 23: 'cook', 24: 'cut', 25: 'dig', 26: 'dress/put on clothing', 27: 'drink', 28: 'drive (e.g., a car, a truck)', 29: 'eat', 30: 'enter', 31: 'exit', 32: 'extract', 33: 'fishing', 34: 'hit (an object)', 35: 'kick (an object)', 36: 'lift/pick up', 37: 'listen (e.g., to music)', 38: 'open (e.g., a window, a car door)', 39: 'paint', 40: 'play board game', 41: 'play musical instrument', 42: 'play with pets', 43: 'point to (an object)', 44: 'press', 45: 'pull (an object)', 46: 'push (an object)', 47: 'put down', 48: 'read', 49: 'ride (e.g., a bike, a car, a horse)', 50: 'row boat', 51: 'sail 

In [39]:
from my_utils.ava_postprocessing import clip_boxes_tensor, map_bbox_from_prep_to_crop, map_bbox_from_crop_to_orig

In [40]:
all_results_dict = {}
for patch_coordinates in tqdm(patches_coordinates, desc='Processing patches'):
    
    # cropping and segmenting input video based on patch_coordinates and temporal window  
    cropped_video = segment_crop_video(video_path, 
                                      frame_index=starting_frame_index, 
                                      length=length_input, 
                                      crop=patch_coordinates)
    
    crop_height, crop_width = cropped_video[0].shape[:2]
    
    
    prep_video = ava_preprocessing_cv2(cropped_video, cfg)
    prep_height, prep_width = prep_video.shape[-2:]
    
    
    prep_clips, center_frames = clip_constructor(prep_video, 
                                                 rate_sample=cfg.DATA.SAMPLING_RATE, 
                                                 num_frames=cfg.DATA.NUM_FRAMES)
    
    list_collated_batches = prepare_collated_batches(prep_clips, center_frames, cfg)
    

    for batch, center_frame_index in tqdm(zip(list_collated_batches, center_frames), 
                                          desc='Processing frames', 
                                          total=len(list_collated_batches)):
        cur_frame_index = center_frame_index + starting_frame_index
        if cur_frame_index not in all_results_dict:
            all_results_dict[cur_frame_index] = []
            
        with torch.no_grad():
            slow_video, fast_video, whwh, boxes, labels, metadata, idx = batch
            clips_height, clips_width = slow_video.shape[-2:]
            slow_video = slow_video.to(device)
            if fast_video is not None:
                fast_video = fast_video.to(device)
            whwh = whwh.to(device)
    
            # INFERENCE
            action_score_list, box_list, objectness_score_list = model(slow_video, fast_video, whwh, boxes, labels)
        
        # Removing batch dimension
        if len(box_list) != 0:
           
            output_bbox = box_list[0]
            output_action = action_score_list[0]
            output_objectness = objectness_score_list[0]
        
            if output_bbox.shape[0] != 0:
            
                # denormalizing bboxes w.r.t. clips shape
                output_bbox_inp = output_bbox.clone()

                output_bbox_inp[:, 0] = output_bbox[:, 0] * clips_width
                output_bbox_inp[:, 1] = output_bbox[:, 1] * clips_height
                output_bbox_inp[:, 2] = output_bbox[:, 2] * clips_width
                output_bbox_inp[:, 3] = output_bbox[:, 3] * clips_height
    
                # clipping bbonx coordinates with prep shape because clip shape is right/bottum padded version of prep shape.
                output_bbox_prep = clip_boxes_tensor(output_bbox_inp, 
                                                     height=whwh[0,1], 
                                                     width = whwh[0,0])
    
                # Scaling bboxes from prep shape to crop shape
                output_bbox_crop = map_bbox_from_prep_to_crop(output_bbox_prep, 
                                                              (crop_height, crop_width), 
                                                              (prep_height, prep_width))
    
                # mapping from crop to original frame
                output_bbox_frame = map_bbox_from_crop_to_orig(output_bbox_crop , patch_coordinates[:2])
                top_values, top_indices = torch.topk(output_action, k=top_k, dim=1)
    
                output_bbox_frame_np = output_bbox_frame.cpu().numpy()
                top_values_np = top_values.cpu().numpy()
                top_indices_np = top_indices.cpu().numpy()
                output_objectness_np = np.reshape(output_objectness.cpu().numpy(), (-1, 1))
    
                agg_resutl = np.concatenate((output_objectness_np, output_bbox_frame_np, top_indices_np, top_values_np), axis=1)
                
                
                all_results_dict[cur_frame_index].append(agg_resutl)
            
            
        
    
    
    
    
    


Processing frames:  13%|████████████████▏                                                                                                           | 42/321 [00:14<01:35,  2.91it/s]


Processing frames:  27%|█████████████████████████████████▏                                                                                          | 86/321 [00:29<01:22,  2.83it/s]


Processing frames:  40%|█████████████████████████████████████████████████▊                                                                         | 130/321 [00:45<01:07,  2.85it/s]


Processing frames:  54%|██████████████████████████████████████████████████████████████████▋                                                        | 174/321 [00:59<00:49,  2.95it/s]


Processing frames:  68%|███████████████████████████████████████████████████████████████████████████████████▌                                       | 218/321 [01:14<00:33,  3.10it/s]


...

Processing frames:  58%|███████████████████████████████████████████████████████████████████████▎                                                   | 186/321 [01:01<00:43,  3.08it/s]


Processing frames:  72%|████████████████████████████████████████████████████████████████████████████████████████▏                                  | 230/321 [01:15<00:29,  3.09it/s]


Processing frames:  85%|████████████████████████████████████████████████████████████████████████████████████████████████████████▉                  | 274/321 [01:30<00:15,  3.08it/s]


Processing frames:  99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊ | 318/321 [01:44<00:00,  3.11it/s]


Processing patches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [17:40<00:00, 117.83s/it]


In [41]:
output_objectness.cpu().numpy().shape, top_indices_np.shape, agg_resutl.shape

((1,), (1, 5), (1, 15))

In [48]:
output_objectness_np.shape, top_indices_np.shape, output_bbox_frame_np.shape, top_values_np.shape

((1, 1), (1, 5), (1, 4), (1, 5))

In [49]:
len(box_list)

1

In [44]:
interesting_actions_indices = [5, 64, 71, 75]
interesting_actions_labels = {5:'fall', 64:'fight', 71:'kick', 75:'push'}

vis_frames_list = []
vis_iteresting_frames_list = []


output_directory_frames = os.path.join(output_directory, 'frames')
os.makedirs(output_directory_frames)


from my_utils.video_processing import get_frame_from_video




# Initialize tqdm with the length of all_results_dict
with tqdm(total=len(all_results_dict)) as pbar:
    for cur_frame, results_frame in all_results_dict.items():
        concatenated_array = np.concatenate(results_frame, axis=0)
        
        obj_scores_frame = concatenated_array[:, :1]
        bboxes_frame = concatenated_array[:, 1:5]
        top_indices_frame = concatenated_array[:, 5:10]
        top_values_frame = concatenated_array[:, 10:]
        
        frame = get_frame_from_video(video_path, cur_frame)
        
        vis_frame = cv2.cvtColor(frame.copy(), cv2.COLOR_BGR2RGB)
        vis_inter_frame = cv2.cvtColor(frame.copy(), cv2.COLOR_BGR2RGB)
        
        id_actor = 0
        interesting_frame = False
        
        for object_score, bbox, top_action_indices, top_action_scores in zip(obj_scores_frame, bboxes_frame, top_indices_frame, top_values_frame):
            x1, y1, x2, y2 = bbox.astype(int)
            small_text = ''
            
            interesting_actor = False
            for ind, act in enumerate(top_action_indices):
                act += 1 # shifting to ava dataset labeling
                if act in interesting_actions_indices:
                    interesting_actor = True
                    interesting_frame = True
                    
                    small_text += '{}_{}'.format(interesting_actions_labels[act], 
                                                 np.round(top_action_scores[ind], 2))
            if interesting_actor:
                long_text = '{}-{}:'.format(id_actor, np.round(object_score, 2)) # add ID of actor for frame visualization
                for act, score in zip(top_action_indices, top_action_scores):
                    long_text += '{}_{}-'.format(label_dict[act+1].replace('(', '').replace(')', '').split('/')[0], 
                                                 (np.round(score, 2)))
                    
            
            if interesting_actor:
                # plot the bbox of interesting actor and adding interesting action
                cv2.rectangle(vis_frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                cv2.putText(vis_frame, small_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                
                cv2.rectangle(vis_inter_frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                #id_text = '{}:{}'.format(id_actor, small_text)
                id_text = '{}'.format(id_actor)
                id_actor += 1
    
                cv2.putText(vis_inter_frame, id_text, (x1+10, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 0, 0), 2)
                cv2.putText(vis_inter_frame, str(cur_frame), (40, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 4)
                
                # add text containing all actions of interesting actor
                if True:
                    cv2.putText(vis_inter_frame, 
                                long_text[:-1], 
                                (20, 100 + 20 * id_actor), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                
                
            else:
                # plot the bbox of other actors
                cv2.rectangle(vis_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.rectangle(vis_inter_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            
        
        #cv2.rectangle(vis_frame, (crop_box[0], crop_box[1]), (crop_box[2], crop_box[3]), (255, 255, 0), 2)
        #cv2.rectangle(vis_inter_frame, (crop_box[0], crop_box[1]), (crop_box[2], crop_box[3]), (255, 255, 0), 2)
        
        
        vis_frames_list.append(vis_frame)
        if interesting_frame:
            vis_iteresting_frames_list.append(vis_inter_frame)
            frame_path = os.path.join(output_directory_frames, f"frame_{cur_frame}.jpg")
            cv2.imwrite(frame_path, cv2.cvtColor(vis_inter_frame, cv2.COLOR_BGR2RGB))
        
        # Update tqdm progress bar
        pbar.update(1)


        
    
    


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 321/321 [03:41<00:00,  1.45it/s]


In [45]:
concatenated_array.shape

(25, 15)

In [46]:
import json

In [47]:
exp_json_path = os.path.join(output_directory, 'exp.json')

# Save the dictionary as a JSON file
with open(exp_json_path, 'w') as f:
    json.dump(exp_dict, f)


In [51]:
pip install json-tricks

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [52]:
import json_tricks as json_tricks

In [56]:
result_json_path = os.path.join(output_directory, 'result.json')

# Save the dictionary as a JSON file
with open(result_json_path, 'w') as f:
    json_tricks.dump(all_results_dict, f, indent=4)

In [57]:
with open(result_json_path, 'r') as f:
    my_dict = json_tricks.load(f)

In [55]:
all_results_dict.keys()

dict_keys([24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 

In [58]:
my_dict.keys()

odict_keys(['24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '160', '161', '162', '163', '164', '165', '166', '167', '168', '169', '170', '171', '172', '173', '174', '175', 

In [59]:
my_dict['24']

[array([[6.72186196e-01, 5.22000000e+02, 2.04000000e+02, 5.61000000e+02,
         2.70000000e+02, 1.10000000e+01, 1.00000000e+01, 1.30000000e+01,
         7.90000000e+01, 1.60000000e+01, 7.47261524e-01, 1.52992934e-01,
         1.35076577e-02, 9.32833087e-03, 6.59528608e-03]]),
 array([[9.76713657e-01, 1.66600000e+03, 3.92000000e+02, 1.74700000e+03,
         5.68000000e+02, 4.50000000e+01, 1.30000000e+01, 7.90000000e+01,
         1.60000000e+01, 7.80000000e+01, 8.66204441e-01, 8.15655887e-01,
         4.37881947e-01, 2.01175079e-01, 1.76363468e-01],
        [9.27072406e-01, 1.78000000e+03, 5.10000000e+02, 1.87600000e+03,
         7.47000000e+02, 1.30000000e+01, 1.60000000e+01, 4.50000000e+01,
         7.90000000e+01, 4.40000000e+01, 9.51462388e-01, 3.57420206e-01,
         3.31895947e-01, 1.34133786e-01, 6.91619962e-02],
        [9.92021263e-01, 1.56700000e+03, 6.34000000e+02, 1.65300000e+03,
         7.96000000e+02, 1.10000000e+01, 1.60000000e+01, 7.90000000e+01,
         7.30000000e+